In [1]:
#hide
import os
import IPython.display as ipd
from pathlib import Path
from ipywidgets import VBox,widgets
import functools
cwd = os.getcwd()

In [2]:
in_path = Path(cwd, 'input')
in_file = Path.joinpath(in_path, "original.mp3")
out_path = Path(cwd, 'separated', 'demucs', 'original')
btn_upload = widgets.FileUpload(accept= '.mp3')
btn_play_in = widgets.Button(description='Play original')
btn_demucs = widgets.Button(description='Demucs')
btn_play_bass = widgets.Button(description='Play bass')
btn_play_drums = widgets.Button(description='Play drums')
btn_play_vocals = widgets.Button(description='Play vocals')
btn_play_other = widgets.Button(description='Play other')
out_pl = widgets.Output()

def on_click_play(change, file=None):
  out_pl.clear_output()
  with out_pl:
    if file:
      out_file = Path.joinpath(out_path, "{}.wav".format(file))
      try:
        display(ipd.Audio(out_file.read_bytes(), autoplay=True))
      except FileNotFoundError:
        print("The file '{}' does not exists.\n".format(file),
              "You should separate the uploaded audio by\n",
              "pressing the demucs button", sep='')
    else:
      try:
        display(ipd.Audio(btn_upload.data[-1], autoplay=True))
      except IndexError:
        print("You should upload a mp3 file")        

def on_click_demucs(change):
  out_pl.clear_output()
  with out_pl:
    in_path.mkdir(exist_ok=True)
    try:
      in_file.write_bytes(btn_upload.data[-1])
      print('Please wait ...')
      !python3 -m demucs.separate --dl -n light -d cpu "./input/original.mp3"
    except IndexError:
      print("You should upload a file")

btn_play_in.on_click(functools.partial(on_click_play, file=None))
btn_demucs.on_click(on_click_demucs)
btn_play_bass.on_click(functools.partial(on_click_play, file='bass'))
btn_play_drums.on_click(functools.partial(on_click_play, file='drums')) 
btn_play_vocals.on_click(functools.partial(on_click_play, file='vocals'))
btn_play_other.on_click(functools.partial(on_click_play, file='other'))

VBox([widgets.Label('Load mp3 file'), 
      btn_upload, btn_play_in, btn_demucs,
      btn_play_drums, btn_play_bass, btn_play_vocals,
      btn_play_other, out_pl])